In [10]:
import os
from langchain_openai import AzureOpenAIEmbeddings
from langchain.embeddings import (
    HuggingFaceEmbeddings,
)
os.environ["AZURE_OPENAI_API_KEY"] = "2b219db0d2984f9dae28b651ab8ab3d9"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://smsh.openai.azure.com/"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-03-01-preview"
# embeddings = AzureOpenAIEmbeddings(
#     model="text-embedding-3-small",
# )

embeddings = HuggingFaceEmbeddings(
                model_name="BAAI/bge-large-en-v1.5",
                # model_kwargs={'device': 'cpu'},
                encode_kwargs={'normalize_embeddings': False},
            )
from langchain.storage import LocalFileStore, RedisStore
from langchain.embeddings import CacheBackedEmbeddings
from langchain_community.vectorstores import FAISS
store = RedisStore(redis_url="redis://localhost:6379")
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
embeddings, store, namespace="bge-large"
)
row_string = []
with open('./data/clean_relations', 'r') as f:
    r_data = [line.strip() for line in f]
all_db = FAISS.from_texts(r_data, cached_embedder)
# retriever = db.as_retriever(search_kwargs={"k": 5})

/tmp/ipykernel_3989324/2109291646.py:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/media/disk1/chatgpt/miniconda3/envs/self-rag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Create with GPT


In [11]:
from langchain_openai import AzureChatOpenAI
import os

os.environ["AZURE_OPENAI_API_KEY"] = "2b219db0d2984f9dae28b651ab8ab3d9"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://smsh.openai.azure.com/"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-02-01"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-35-turbo"

model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    temperature=1,
    n = 3,
    max_retries=5, request_timeout=600
)
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.chains import LLMChain

In [12]:
# The name of Justin Bieber's brother is Jaxon Bieber. This is based on the reasoning path that connects Justin Bieber to Jaxon Bieber through the relationship of sibling. The other paths that connect Justin Bieber to Jazmyn Bieber or back to Justin Bieber himself are incorrect in this context.
few_shot_intepretable_prompt = FewShotPromptTemplate(
        examples=[{
            "query": "Who was the prime minister of Japan in 2011, that served in the New Party Sakigake?",
            "evidence": "Relations retrieved: language.human_language.countries_spoken_in\n Entities retrieved: Japan",
            "preceding_sentences": "",
            "output": "Naoto Kan",
            "rating": "[Continue to Retrieve Evidence]"
        }],
        example_prompt=PromptTemplate.from_template("""Query: {query}
Preceding sentences: {preceding_sentences}
Evidence: {evidence}
Output: {target_output}
Rating: {rating}"""),
        prefix=
        "You will be provided with a query, evidence, output sentence, and preceding sentences (optional). Your task is to determine whether the information in the output sentence can be fully verified by the evidence or if it requires further external verification. There are three cases:\n" 
        "- If the output sentence can be verified solely with the evidence and the preceding sentences, then respond with [No Retrieval]. \n"
        "- If additional information about the tail entity in evidence is needed to verify the output sentence, respond with [Continue to Retrieve Evidence] \n"
        "- If more information unrelated to the evidence is needed, e.g. totally new relationship or new entity, reponse with [New Retrieval].\n\n",
        suffix=
        "Query: {query}\n"
        "Preceding sentences: {preceding_sentences}\n"
        "Evidence: {evidence}\n"
        "Output: {target_output}",
        input_variables=["query", "evidence", "preceding_sentences", "topic"],
)

llm_chain = LLMChain(llm=model, prompt=few_shot_intepretable_prompt, verbose=True)


/tmp/ipykernel_3989324/3707533165.py:28: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=model, prompt=few_shot_intepretable_prompt, verbose=True)


In [30]:
# relationship relevance
graph_intepretable =  """You will receive a query, evidence and optional preceding sentences containing history information. The evidence contains graph relationships or entities may be useful to answer the query. Your task is to filters out valid information from the evidence to answer the given query, evaluate your output and provide explanations on your result.

###
Query: Name the president of the country whose main spoken language was Brahui in 1980?
Topic Entity: Brahui Language
Evidence: language.human_language.main_country; language.human_language.language_family; language.human_language.iso_639_3_code; base.rosetta.languoid.parent; language.human_language.writing_system; base.rosetta.languoid.languoid_class; language.human_language.countries_spoken_in; kg.object_profile.prominent_type; base.rosetta.languoid.document; base.ontologies.ontology_instance.equivalent_instances; base.rosetta.languoid.local_name; language.human_language.region
Preceding sentences: 
Output: 
1. {{language.human_language.main_country (Score: Fully relavant))}}: This relation is highly relevant as it directly relates to the country whose president is being asked for, and the main country where Brahui language is spoken in 1980.
2. {{language.human_language.countries_spoken_in (Score: Fully relavant)}}: This relation is also relevant as it provides information on the countries where Brahui language is spoken, which could help narrow down the search for the president.
3. {{base.rosetta.languoid.parent (Score: Partially relevant)}}: This relation is less relevant but still provides some context on the language family to which Brahui belongs, which could be useful in understanding the linguistic and cultural background of the country in question.

###
Query: {query}
Topic Entity: {topic}
Evidence: {evidence}
Preceding sentences: {preceding_sentences}
Output: 
"""
# The name of Justin Bieber's brother is Jaxon Bieber. This is based on the reasoning path that connects Justin Bieber to Jaxon Bieber through the relationship of sibling. The other paths that connect Justin Bieber to Jazmyn Bieber or back to Justin Bieber himself are incorrect in this context.
few_shot_intepretable_prompt = FewShotPromptTemplate(
        examples=[{
            "query": "Name the president of the country whose main spoken language was Brahui in 1980?",
            "topic": "Brahui Language",
            "evidence": "language.human_language.main_country; language.human_language.language_family; language.human_language.iso_639_3_code; base.rosetta.languoid.parent; language.human_language.writing_system; base.rosetta.languoid.languoid_class; language.human_language.countries_spoken_in; kg.object_profile.prominent_type; base.rosetta.languoid.document; base.ontologies.ontology_instance.equivalent_instances; base.rosetta.languoid.local_name; language.human_language.region",
            "preceding_sentences": "",
            "output": """1. {{language.human_language.main_country (Score: [Fully Relavant])}}: This relation is highly relevant as it directly relates to the country whose president is being asked for, and the main country where Brahui language is spoken in 1980.
2. {{language.human_language.countries_spoken_in (Score: [Fully Relavant])}}: This relation is also relevant as it provides information on the countries where Brahui language is spoken, which could help narrow down the search for the president.
3. {{base.rosetta.languoid.parent (Score: [Partially Relevant])}}: This relation is less relevant but still provides some context on the language family to which Brahui belongs, which could be useful in understanding the linguistic and cultural background of the country in question."""
        }],
        example_prompt=PromptTemplate.from_template("""###
Query: {query}
Topic Entity: {topic}
Evidence: {evidence}
Preceding sentences: {preceding_sentences}
Output: {output}
"""),
        prefix=
        """You will receive a query, topic entity, evidence and optional preceding sentences containing history information. The evidence contains graph relationships or entities may be useful to answer the query. Your task is to filters out 3 valid information from the evidence that contribute to answering the query and provide a relevance score for each output, output your explanations for the score.
The score of relevance range from [Fully Relavant], [Partially Relevant] to [Unrelevant].""",
        suffix=
        """###
Query: {query}
Topic Entity: {topic}
Evidence: {evidence}
Preceding sentences: {preceding_sentences}
Output: """,
        input_variables=["query", "evidence", "preceding_sentences", "topic"],
)
graph_intepretable_prompt = PromptTemplate(input_variables=["query", "evidence", "preceding_sentences", "topic"], template=
graph_intepretable)
llm_chain = LLMChain(llm=model, prompt=few_shot_intepretable_prompt, verbose=True)


In [31]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.chains import LLMChain
few_shot_intepretable_prompt = FewShotPromptTemplate(
        examples=[{
            "query": "what is the name of justin bieber brother?",
            "output": "[Retreive New Relationship]<paragraph>people.sibling_relationship.sibling;fictional_universe.fictional_character.siblings;fictional_universe.sibling_relationship_of_fictional_characters.siblings;people.person.sibling_s;people.family.members;people.person.parents</paragraph>Retrieved relationship: people.person.parents[Fully Relevant][Retrieve Entity]<paragraph>(Justin Bieber, people.person.parents, Pattie Mallette);(Justin Bieber, people.person.parents, Jeremy Bieber);(Justin Bieber, people.person.parents, Jeremy Bieber)</paragraph>Retrieved triplet: (Justin Bieber, people.person.parents, Jeremy Bieber)[Fully Relevant][Continue to Retrieve Evidence]<paragraph>people.sibling_relationship.sibling;people.person.sibling_s;people.person.parents;fictional_universe.fictional_character.siblings;people.family.members;people.person.children</paragraph>Retrieved relationship: people.person.children[Fully Relevant][Retrieve Entity]<paragraph>(Jeremy Bieber, people.person.children, Jazmyn Bieber);(Jeremy Bieber, people.person.children, Justin Bieber);(Jeremy Bieber, people.person.children, Jaxon Bieber);(Jeremy Bieber, people.person.children, Jaxon Bieber)</paragraph>Retrieved triplet: (Jeremy Bieber, people.person.children, Jaxon Bieber)[Fully Relevant][No Retrieval] Answer: Jaxon Bieber",
            "explanation": "The output provides the name of justin bieber's brother. This is based on the reasoning path that connects Justin Bieber to Jaxon Bieber through the relationship of sibling. The other paths that connect Justin Bieber to Jazmyn Bieber or back to Justin Bieber himself are incorrect in this context.",
            "rating": "[Confidence:5]"
        }],
        example_prompt=PromptTemplate.from_template("""
Query: {query}\n
Output: {output}
Explanation: {explanation}
Rating: {rating}
"""),
        prefix="""Given a query and an output, rate whether the response and the thought process appears to be a helpful and informative answer to the query, from 1 (lowest) - 5 (highest). We call this confidence score.
[Confidence:5]: The response provides a complete and correct reasoning chain to the query, and the final answer is complete and logically correct.
[Confidence:4]: The response mostly fulfills the need in the query and provides correct answers, while there can be some minor improvements such as shorter reasoning chain or less repetition.
[Confidence:3]: The response is acceptable, but the answers may be not complete or needs minor improvement.
[Confidence:2]: The reasoning process still addresses the main request, but the answers are not correct or not relevant to the query.
[Confidence:1]: The reasoning is barely irrelevant or does not give an answer in the end.""",
        suffix=
        """
Query: {query}\n
Output: {output}\n""",
        input_variables=["query", "output"],
)
# confidence_prompt = PromptTemplate(input_variables=["query", "output"], template=
# graph_intepretable)
llm_chain = LLMChain(llm=model, prompt=few_shot_intepretable_prompt, verbose=True)


### Run_long_form answer

In [33]:
from vllm import LLM, SamplingParams
# model = LLM(model='/media/disk2/llama_factory/generation_1124_special', trust_remote_code=True, tensor_parallel_size=4)
model = LLM(model='/media/disk2/llama_factory/generation_1209_no_mask', trust_remote_code=True, tensor_parallel_size=4)



WARNING 12-19 06:20:03 cuda.py:22] You are using a deprecated `pynvml` package. Please install `nvidia-ml-py` instead, and make sure to uninstall `pynvml`. When both of them are installed, `pynvml` will take precedence and cause errors. See https://pypi.org/project/pynvml for more information.


2024-12-19 06:20:03,921	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 12-19 06:20:08 config.py:905] Defaulting to use mp for distributed inference
INFO 12-19 06:20:08 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='/media/disk2/llama_factory/generation_1209_no_mask', speculative_config=None, tokenizer='/media/disk2/llama_factory/generation_1209_no_mask', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, ser

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:00,  3.59it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.60it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.90it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.13it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.06it/s]



(VllmWorkerProcess pid=4047129) INFO 12-19 06:20:17 model_runner.py:1067] Loading model weights took 3.7420 GB
INFO 12-19 06:20:17 model_runner.py:1067] Loading model weights took 3.7420 GB
(VllmWorkerProcess pid=4047128) INFO 12-19 06:20:17 model_runner.py:1067] Loading model weights took 3.7420 GB
(VllmWorkerProcess pid=4047130) INFO 12-19 06:20:17 model_runner.py:1067] Loading model weights took 3.7420 GB
INFO 12-19 06:20:20 distributed_gpu_executor.py:57] # GPU blocks: 61347, # CPU blocks: 8192
INFO 12-19 06:20:20 distributed_gpu_executor.py:61] Maximum concurrency for 8192 tokens per request: 119.82x
(VllmWorkerProcess pid=4047130) INFO 12-19 06:20:22 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
(VllmWorkerProcess pid=4047130) INFO 12-19 06:20:22 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB

In [34]:
#是否带special token 分开传入，用于检索和生成
sampling_params = SamplingParams(
            temperature=0.01, top_p=1.0,max_tokens=100, logprobs=5, skip_special_tokens=False, include_stop_str_in_output=True)
PROMPT_DICT = {"llama3": '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\n{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>'}
model.generate([PROMPT_DICT["llama3"].format(input="what all does google now do?")], sampling_params)[0].outputs[0]

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.01s/it, est. speed input: 16.91 toks/s, output: 99.50 toks/s]


CompletionOutput(index=0, text='business[Partially Relevant]organization.organization_founder.organizations_founded;organization.organization.founders;organization.organization_membership.organization;organization.leadership.organization;business.sponsorship.sponsored_by</paragraph>organization.organization_membership.organization[Fully Relevant]organization.organization_founder.organizations_founded[Partially Relevant]organization.leadership.organization[Partially Relevant][Retrieve Entity]<paragraph>(Google, organization.organization.founders, Sergey Brin);(Google, organization.organization.founders, Larry Page)</paragraph>Sergey Brin[Partially Relevant]Larry Page[Partially Relevant][Continue to Retrieve Evidence]<paragraph>people.person.profession;', token_ids=(27243, 128262, 24844, 70324, 22200, 261, 51272, 8200, 766, 13382, 26, 24844, 70324, 840, 801, 388, 26, 24844, 70324, 85735, 70324, 26, 24844, 31602, 6527, 2200, 70324, 26, 27243, 516, 31341, 5383, 516, 35841, 3795, 128264, 24

In [35]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('/media/disk2/llama_factory/generation_1124_special')

In [36]:
def load_special_tokens(tokenizer, use_grounding=False, use_utility=False):
    rel_tokens = {}
    for token in ['[Unrelevant]','[Partially Relevant]','[Fully Relevant]']:
        rel_tokens[token] = tokenizer.convert_tokens_to_ids(token)

    # ut_tokens = None
    # if use_utility is True:
    #     ut_tokens = {}
    #     for token in utility_tokens_names:
    #         ut_tokens[token] = tokenizer.convert_tokens_to_ids(token)

    return rel_tokens

In [37]:
rel_tokens = load_special_tokens(tokenizer)

In [38]:
import numpy as np
import re
from src.sparql_utils import *
def run_step_generation_batch(model, prompt, topic_entity,new_retrieval, beam_width=3):
    pattern = r'(.*?)\[(.*?)\]'
    rel_score_dict = {}
    return_entities = []
    final_preds = []
    overall_scores = {}
    paragraph = ';'.join([page.page_content.strip() for page in retriever.invoke(prompt.split('\n\n')[1].split('<|eot_id|>')[0])])
    print(paragraph)
    if new_retrieval:
        retrieval_token = "[New Retrieval]"
        aug_prompts =  ["<paragraph>{}</paragraph>".format(paragraph)]
    else:
        retrieval_token = "[Continue to Retrieve Evidence]"
        aug_prompts =  ["<paragraph>{}</paragraph>".format(paragraph)]
    
    pred = model.generate(prompt + retrieval_token + aug_prompts[0], sampling_params)[0]
    pred_token_ids = pred.outputs[0].token_ids
    pred_text_1 = pred.outputs[0].text
    pred_log_probs = pred.outputs[0].logprobs
    seq_score = pred.outputs[0].cumulative_logprob / \
        max(len(pred.outputs[0].token_ids), 1)
    relevance_indices = []
    for tok_idx, tok in enumerate(pred_token_ids):
        if tok in rel_tokens.values():
            relevance_indices.append(tok_idx)
    if len(relevance_indices) > 0:
        for idx in relevance_indices:
            for token, token_id in rel_tokens.items():
                prob = pred_log_probs[idx][token_id].logprob if token_id in pred_log_probs[idx] else -100
                rel_score_dict[token] = np.exp(prob)
    relevance_score = rel_score_dict['[Fully Relevant]']+ rel_score_dict['[Partially Relevant]'] / np.sum(list(rel_score_dict.values()))
    processed_pred = pred_text_1.split('[Retrieve Entity]')[0]
    matches =  dict(re.findall(pattern,processed_pred))
    
    name2id = dict()
    entity_prompts = []
    for _, entity in enumerate(topic_entity):
        entities = []
        for k, v in matches.items():
            if v in ['Fully Relevant', 'Partially Relevant']:
                another_entities = get_another_entity(entity, k, return_label=True)
                # print(another_entities)
                name2id.update(another_entities)
                entities.extend([f'({get_label(entity)}, {k}, {e})' for e in another_entities.values()])
        entity_prompts.append(aug_prompts[0] + processed_pred +  '[Retrieve Entity]' + "<paragraph>{}</paragraph>".format(';'.join(entities[:10])))
    # print(aug_prompts)
    preds = model.generate([prompt + retrieval_token+ entity_prompts[i] for i in range(len(entity_prompts))], sampling_params)
    
    for p_idx, pred in enumerate(preds):
        pred_token_ids = pred.outputs[0].token_ids
        pred_text_2 = pred.outputs[0].text
        pred_log_probs = pred.outputs[0].logprobs
        rel_score_dict = {}
        relevance_indices = []
        for tok_idx, tok in enumerate(pred_token_ids):
            if tok in rel_tokens.values():
                relevance_indices.append(tok_idx)
        if len(relevance_indices) > 0:
            # print(relevance_indices)
            for idx in relevance_indices:
                for token, token_id in rel_tokens.items():
                    prob = pred_log_probs[idx][token_id].logprob if token_id in pred_log_probs[idx] else -100
                    rel_score_dict[token] = np.exp(prob)
        overall_scores[p_idx] = relevance_score + rel_score_dict['[Fully Relevant]'] + rel_score_dict['[Partially Relevant]']/ np.sum(list(rel_score_dict.values()))
        if '[Continue to Retrieve Evidence]' in pred_text_2:
            processed_pred = pred_text_2.split('[Continue to Retrieve Evidence]')[0]
            matches =  dict(re.findall(pattern, processed_pred))
            for k, v in matches.items():
                if v in ['Fully Relevant', 'Partially Relevant']:
                    if k in name2id:
                        return_entities.append(name2id[k])
            processed_pred += '[Continue to Retrieve Evidence]'
        else:
            processed_pred = pred_text_2
        final_preds.append(aug_prompts[0] + processed_pred)
    return final_preds, [overall_scores[p_idx] for p_idx in overall_scores], return_entities

        

In [96]:
import json
with open('./data/merged/WebQSP_test.json', 'r',encoding='utf-8') as f:
    test_data = json.load(f)
print(len(test_data))

1639


In [39]:
import json
with open('./data/merged/CWQ_test.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)
print(len(test_data))

3531


In [ ]:
def backtracking_prediction_tree(levels: dict[int,list[int]], curr_depth: int, prediction_tree: dict[int, dict]) -> dict[int,list[int]]:
    '''
    get best tracking from prediction_tree base on levels
    '''
    parent = 0 
    best_selections = {}
    # Traverse from the bottom 
    levels = {k: v for k, v in levels.items() if len(v) > 0 and k != 0} # remove empty list in levels
    for path_i, node in enumerate(levels[len(levels)]): # beam search 
        if node == 0:
            break
        best_selections[path_i] = [node] 
        current_node = node 
        current_level = curr_depth 
        if current_node is None:
            continue
        while current_level > 0 and current_node is not None:
            parent = prediction_tree[current_node]["parent"]
            best_selections[path_i] = [parent] + best_selections[path_i] 
            current_node = parent 
            current_level -= 1
    return best_selections


In [ ]:
best_selections = backtracking_prediction_tree(levels, curr_depth, prediction_tree)
final_prediction = {}
splitted_sentences = {}
original_splitted_sentences = {}
ctxs = {}
for path_i, nodes in best_selections.items():
    final_prediction[path_i] = " ".join([prediction_tree[node]["processed_pred"] for node in nodes if node is not None])

### Tree fix

In [68]:
from typing import List
import random
def save_to_json(data: List, data_path=''):
    if not os.path.isfile(data_path):
        # 文件不存在，创建新列表并写入文件
        with open(data_path, 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=4)
        return
    try:
        # 尝试读取现有文件
        with open(data_path, 'r', encoding='utf-8') as file:
            # 加载现有的JSON数据
            existing_data = json.load(file)
            existing_data.extend(data)
    except json.JSONDecodeError:
        # 文件不是有效的JSON，打印错误信息并退出
        print(f"文件 {data_path} 不是有效的JSON格式。")
        return
    except ValueError as e:
        # 打印错误信息并退出
        print(e)
        return
    # 将更新后的数据写回文件
    with open(data_path, 'w', encoding='utf-8') as file:
        json.dump(existing_data, file, ensure_ascii=False, indent=4)
def random_sample(lst, k=3):
    try:
        # 尝试从列表中随机抽取k个不重复的元素
        return random.sample(lst, k)
    except ValueError:
        # 如果列表长度小于k，返回整个列表
        return lst

In [69]:
import numpy as np
import re
from src.sparql_utils import *
import random
def run_relation_generation_batch(model, prompt, new_retrieval, context, topic_entity, hypo=True, use_1hop=True):
    rel_score_dict = {}
    final_preds = []
    overall_scores = []
    final_contexts = []
    if new_retrieval:
        retrieval_token = "[New Retrieval]"
    else:
        retrieval_token = "[Continue to Retrieve Evidence]"
    if use_1hop:
        candidate_relations = []
        for entity in topic_entity:
            try:
                candidate_relations.extend(get_1hop_relations_with_odbc(entity))
            except:
                continue
        if len(candidate_relations):
            vec_db = FAISS.from_texts(candidate_relations, cached_embedder)
        else:
            vec_db = all_db
        retriever = vec_db.as_retriever(search_kwargs={"k": 5})
    else:
        retriever = all_db.as_retriever(search_kwargs={"k": 5})
    paragraph = [page.page_content.strip() for page in retriever.invoke(prompt.split('\n\n')[1].split('<|eot_id|>')[0] + ' '+ context)]
    if hypo:
        hypo_rel = model.generate(prompt + retrieval_token, sampling_params)[0].outputs[0].text
        pattern = r'(\w+\.\w+\.\w+)\[(.*?)\]'
        if '[Retrieve Entity]' in hypo_rel:
            hypo_rel = hypo_rel.split('[Retrieve Entity]')[0]
        matches =  dict(re.findall(pattern, hypo_rel))
        string = ''
        for k,v in matches.items():
            if v in ['Fully Relevant', 'Partially Relevant']:
                string += k + ' '
        for extra_rel in retriever.invoke(string):
            if extra_rel.page_content.strip() not in paragraph:
                paragraph.append(extra_rel.page_content.strip())
    
    aug_prompts =  ["<paragraph>{}</paragraph>".format(';'.join(p))  for p in [paragraph[i: i+5] for i in range(0, len(paragraph), 5)]]
    
    preds = model.generate([prompt + retrieval_token + aug for aug in aug_prompts], sampling_params)
    for p_id, pred in enumerate(preds):
        pred_token_ids = pred.outputs[0].token_ids
        pred_text_1 = pred.outputs[0].text
        pred_log_probs = pred.outputs[0].logprobs
        seq_score = pred.outputs[0].cumulative_logprob / \
            max(len(pred.outputs[0].token_ids), 1)
        relevance_indices = []
        for tok_idx, tok in enumerate(pred_token_ids):
            if tok in rel_tokens.values():
                relevance_indices.append(tok_idx)
        if len(relevance_indices) > 0:
            for idx in relevance_indices:
                for token, token_id in rel_tokens.items():
                    prob = pred_log_probs[idx][token_id].logprob if token_id in pred_log_probs[idx] else -100
                    rel_score_dict[token] = np.exp(prob)
        relevance_score = rel_score_dict['[Fully Relevant]']+ rel_score_dict['[Partially Relevant]'] / np.sum(list(rel_score_dict.values()))
        if '[Retrieve Entity]' in pred_text_1:
            processed_pred = pred_text_1.split('[Retrieve Entity]')[0] + '[Retrieve Entity]'
        else:
            processed_pred = pred_text_1
        final_preds.append(retrieval_token + aug_prompts[p_id] + processed_pred)
        overall_scores.append(relevance_score)
        final_contexts.append(pred_text_1.split('[Retrieve Entity]')[0])

    return final_preds, overall_scores, final_contexts

def run_entity_generation_batch(model, prompt, topic_entity, context, score_type='hard'):
    final_preds = []
    overall_scores = {}
    final_entities = []
    final_context = []
    pattern = r'(.*?)\[(.*?)\]'
    matches =  dict(re.findall(pattern,context))
    name2id = {}
    effective_count = 0
    entity_prompts = []
    for _, entity in enumerate(topic_entity):
        for k, v in matches.items():
            if v in ['Fully Relevant', 'Partially Relevant']:
                entities = []
                try:
                    another_entities = get_another_entity(entity, k, return_label=True)
                except:
                    another_entities = []
                # handle Unkown mid entities
                # whether mix another entities
                if len(another_entities):
                    name2id.setdefault(effective_count, {})
                    name2id[effective_count].update(another_entities)
                    effective_count += 1
                    entities.extend([f'({get_label(entity)}, {k}, {e})' for e in another_entities.keys()])
                    entity_prompts.append("<paragraph>{}</paragraph>".format(';'.join(entities[:5])))
    # print(aug_prompts)
    preds = model.generate([prompt+  '[Retrieve Entity]' + entity_prompts[i] for i in range(len(entity_prompts))], sampling_params)
    
    for p_idx, pred in enumerate(preds):
        return_entities = dict()
        pred_token_ids = pred.outputs[0].token_ids
        pred_text_2 = pred.outputs[0].text
        pred_log_probs = pred.outputs[0].logprobs
        # hard decode 分数
        rel_score = 0
        # rel_score_dict = {}
        # relevance_indices = []
        # for tok_idx, tok in enumerate(pred_token_ids):
        #     if tok in rel_tokens.values():
        #         relevance_indices.append(tok_idx)
        # if len(relevance_indices) > 0:
        #     # print(relevance_indices)
        #     for idx in relevance_indices:
        #         for token, token_id in rel_tokens.items():
        #             prob = pred_log_probs[idx][token_id].logprob if token_id in pred_log_probs[idx] else -100
        #             rel_score_dict[token] = np.exp(prob)
        # if len(rel_score_dict) == 3:
        #     overall_scores[p_idx] = rel_score_dict['[Fully Relevant]'] + rel_score_dict['[Partially Relevant]']/ np.sum(list(rel_score_dict.values()))
        if '[Continue to Retrieve Evidence]' in pred_text_2:
            processed_pred = pred_text_2.split('[Continue to Retrieve Evidence]')[0]
            matches =  dict(re.findall(pattern, processed_pred))
            for k, v in matches.items():
                if v in ['Fully Relevant', 'Partially Relevant']:
                    if k in name2id[p_idx]:
                        return_entities[k] = name2id[p_idx][k]
                    elif k == 'Unknown Entity':
                        random_key = random_sample(list(name2id[p_idx].keys()))
                        return_entities[random_key] = name2id[p_idx][random_key]
                    rel_score += 1 if v =='Fully Relevant' else 0.5
            processed_pred += '[Continue to Retrieve Evidence]'

        elif '[No Retrieval]' in pred_text_2:
            processed_pred = pred_text_2
            rel_score += 100
        else:
            processed_pred = '[No Retrieval]'
        overall_scores[p_idx] = rel_score
        final_preds.append('[Retrieve Entity]' + entity_prompts[p_idx]+processed_pred)
        final_entities.append(list(return_entities.values()))
        final_context.append(' '.join(return_entities.keys()))
    return final_preds, [overall_scores[p_idx] for p_idx in overall_scores], final_entities, final_context
 
        

In [104]:
index

3208

In [60]:
count = 0
correct_ids = []
logging_res = []
for index in range(21, len(test_data)):
# index = 483
    hit = 0
    print(f'Process {index}')
    data_input = test_data[index]['question']
    prompt = PROMPT_DICT['llama3'].format(input= data_input)
    max_depth = 9
    topic_entity = list(test_data[index]['gold_entity_map'].keys())
    # pred = model.generate([prompt], sampling_params)[0]
    # pred_text = pred.outputs[0].text
    # if '[New Retrieval]' in pred_text:
    curr_depth = 1
    terminated = False
    node_id = 0
    prediction_tree = {}
    levels = {}
    prediction_tree[node_id] = {"prompt": prompt, "pred": "[New Retrieval]",
                                "processed_pred": "", "score": None, "topic_entity": topic_entity, "parent": None, "context": ''}
    levels[0] = [0]
    while curr_depth < max_depth:
        levels[curr_depth] = []
        if curr_depth-1 in levels:
            for node in levels[curr_depth-1]:
                curr_pred = prediction_tree[node]["pred"]
                if "<|eot_id|>" in curr_pred:
                    continue
                prompt = prediction_tree[node]["prompt"]
                prev_generation = prediction_tree[node]["processed_pred"]
                score = prediction_tree[node]["score"]
                topic_entity = prediction_tree[node]["topic_entity"]
                context = prediction_tree[node]['context']
                cur_prompt = prompt + prev_generation
                if "Retrieve Entity" in curr_pred.split('[')[-1]:
                    retrieval_results = {}
                    preds, scores, next_entities, contexts = run_entity_generation_batch(
                        model, cur_prompt, topic_entity, context)
                    for i, (pred, p_score,next_topic, context) in enumerate(zip(preds, scores, next_entities, contexts)):
                        retrieval_results[i] = {
                            "pred": pred, "score": p_score, "next_topic": next_topic, "context": context}

                    for i, result in retrieval_results.items():
                        node_id += 1
                        node_score = result["score"] * \
                            score if score is not None else result["score"]
                        pred = result["pred"]
                        next_entity = result['next_topic']
                        if len(next_entity) == 0:
                            next_entity = topic_entity
                        prediction_tree[node_id] = {"prompt": cur_prompt, "pred": pred, "context": result['context'],
                                                    "score": node_score, "parent": node,
                                                    "topic_entity": next_entity}
                        if "[Continue to Retrieve Evidence]" in pred:
                            gen_result_index = pred.index("[Continue to Retrieve Evidence]")
                            prev_generation = pred[:gen_result_index]
                        else:
                            prev_generation = pred
                        prediction_tree[node_id]["processed_pred"] = prev_generation
                        levels[curr_depth].append(node_id)
                #存在前后逻辑粘连   
                if "New Retrieval" in curr_pred.split('[')[-1] or "Continue to Retrieve Evidence" in curr_pred.split('[')[-1]:
                    retrieval_results = {}
                    preds, scores, contexts = run_relation_generation_batch(
                        model, cur_prompt, new_retrieval=True if ("[New Retrieval]" in curr_pred) else False, context=context, topic_entity=topic_entity, hypo=True)
                    for i, (pred, p_score, context) in enumerate(zip(preds, scores, contexts)):
                        retrieval_results[i] = {
                            "pred": pred, "score": p_score, "context": context}

                    for i, result in retrieval_results.items():
                        node_id += 1
                        node_score = result["score"] * \
                            score if score is not None else result["score"]
                        pred = result["pred"]
                        context = result["context"]
                        prediction_tree[node_id] = {"prompt": cur_prompt, "pred": pred,
                                                    "score": node_score, "parent": node,
                                                    "topic_entity": topic_entity, "context": context}
                        if "[Retrieve Entity]" in pred:
                            gen_result_index = pred.index("[Retrieve Entity]")
                            prev_generation = pred[:gen_result_index]
                        else:
                            prev_generation = pred
                        prediction_tree[node_id]["processed_pred"] = prev_generation
                        levels[curr_depth].append(node_id)
            current_rank = levels[curr_depth]
            node2score = {
                node_id: prediction_tree[node_id]["score"] for node_id in current_rank}
            top_nodes = sorted(node2score.items(), key=lambda x: x[1], reverse=True)[
                :3]
            levels[curr_depth] = [node[0] for node in top_nodes]
            curr_depth += 1
        else:
            break
    labels = [get_label(ans) if ans.startswith('m.') else ans for ans in test_data[index]['answer']]
    # print(labels)
    for tree_node in prediction_tree.values():
        if 'Answer' in tree_node['processed_pred']:
            answer = tree_node['processed_pred'].split('Answer:')[-1]
            for label in labels:
                if label and label in answer:
                    hit = 1
    if hit == 1:
        print('Correct')
        count += 1
        correct_ids.append(index)
    else:
        break
        logging_res.append({"index": index, "tree": prediction_tree})
    if len(logging_res) == 20:
        save_to_json(logging_res, './output/inference/cwq_test_res_1217.json')
        logging_res = []
#1217修改了relation-1hop,修改了score
save_to_json(logging_res, './output/inference/cwq_test_res_1217.json')
    #注意有value标签, e.g. WebQTest-31
    # for label in labels:
    #     if label and label in prediction_tree[len(prediction_tree)-1]['processed_pred']:
    #         count += 1
    #         break
    # except:
    #     print(f'{index} Error')
    #     continue


Process 21


Processed prompts: 100%|██████████| 3/3 [00:00<00:00,  3.43it/s, est. speed input: 842.48 toks/s, output: 151.30 toks/s]


In [16]:
with open('./output/inference/correct_cwq.json', 'w') as f:
    json.dump(correct_ids, f)

In [61]:
levels

{0: [0], 1: [1, 2], 2: [3], 3: [4, 5], 4: [6, 8, 7]}

In [55]:
get_another_entity('m.0j5wjnc', 'type.type.instance')

{'Abstract': 'base.type_ontology.abstract',
 'Non-Agent': 'base.type_ontology.non_agent',
 'Topic': 'common.topic',
 'Government Office or Title': 'government.government_office_or_title'}

In [70]:
prediction_tree

{0: {'prompt': '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nWhich country that is situated in the ASEAN Common Time Zone has the largest population?<|eot_id|><|start_header_id|>assistant<|end_header_id|>',
  'pred': '[New Retrieval]',
  'processed_pred': '',
  'score': None,
  'topic_entity': ['m.01mp', 'm.0bzt6c'],
  'parent': None,
  'context': ''},
 1: {'prompt': '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nWhich country that is situated in the ASEAN Common Time Zone has the largest population?<|eot_id|><|start_header_id|>assistant<|end_header_id|>',
  'pred': '[New Retrieval]<paragraph>location.location.time_zones;time.time_zone.time_zone_name_standard;time.time_zone.time_zone_abbreviation_standard;time.time_zone.locations_in_this_time_zone;time.time_zone.offset_from_uct</paragraph>time.time_zone.locations_in_this_time_zone[Fully Relevant]location.location.time_zones[Partially Relevant]time.time_zone.time_zone_name_standard[Unrelevant][Retrieve Ent

In [95]:

get_another_entity('m.027rn', 'location.location.time_zones')


{'UTC−04:00': 'm.0730gz', 'Atlantic Time Zone': 'm.042g7t'}

In [ ]:
def cal_eval_metric(best_pred, preds, answers):
    correct, total = 0.0, 0.0
    for entity in preds:
        if entity in answers:
            correct += 1
        total += 1
    if len(answers) == 0:
        if total == 0:
            return 1.0, 1.0, 1.0, 1.0 # precision, recall, f1, hits
        else:
            return 0.0, 1.0, 0.0, 1.0 # precision, recall, f1, hits
    else:
        hits = float(best_pred in answers)
        if total == 0:
            return 1.0, 0.0, 0.0, hits # precision, recall, f1, hits
        else:
            precision, recall = correct / total, correct / len(answers)
            f1 = 2.0 / (1.0 / precision + 1.0 / recall) if precision != 0 and recall != 0 else 0.0
            return precision, recall, f1, hits

In [ ]:
def cal_eval_metric(preds, answers):
    correct, total = 0.0, 0.0
    for entity in preds:
        if entity in answers:
            correct += 1
        total += 1
    # if len(answers) == 0:
    #     if total == 0:
    #         return 1.0, 1.0, 1.0, 1.0 # precision, recall, f1, hits
    #     else:
    #         return 0.0, 1.0, 0.0, 1.0 # precision, recall, f1, hits
    # else:
    if total != 0:
        hits = 1
    else: 
        hits = 0
    if total == 0:
        return 1.0, 0.0, 0.0, hits # precision, recall, f1, hits
    else:
        precision, recall = correct / total, correct / len(answers)
        f1 = 2.0 / (1.0 / precision + 1.0 / recall) if precision != 0 and recall != 0 else 0.0
        return precision, recall, f1, hits

In [ ]:
cal_eval_metric(preds, answers)

(1.0, 0.3333333333333333, 0.5, 1)

In [ ]:
# count = 0
# correct_ids = []
# logging_res = []
# for index in range(21, len(test_data)):
# # index = 483
#     hit = 0
#     print(f'Process {index}')
#     data_input = test_data[index]['question']
#     prompt = PROMPT_DICT['llama3'].format(input= data_input)
#     max_depth = 5
#     topic_entity = list(test_data[index]['gold_entity_map'].keys())
#     # pred = model.generate([prompt], sampling_params)[0]
#     # pred_text = pred.outputs[0].text
#     # if '[New Retrieval]' in pred_text:
#     curr_depth = 1
#     terminated = False
#     node_id = 0
#     prediction_tree = {}
#     levels = {}
#     prediction_tree[node_id] = {"prompt": prompt, "pred": "[New Retrieval]",
#                                 "processed_pred": "", "score": None, "topic_entity": topic_entity, "parent": None, "context": ''}
#     levels[0] = [0]
#     while curr_depth < max_depth:
#         levels[curr_depth] = []
#         if curr_depth-1 in levels:
#             for node in levels[curr_depth-1]:
#                 curr_pred = prediction_tree[node]["pred"]
#                 if "<|eot_id|>" in curr_pred:
#                     continue
#                 prompt = prediction_tree[node]["prompt"]
#                 prev_generation = prediction_tree[node]["processed_pred"]
#                 score = prediction_tree[node]["score"]
#                 topic_entity = prediction_tree[node]["topic_entity"]
#                 context = prediction_tree[node]['context']
#                 cur_prompt = prompt + prev_generation
#                 if "Retrieve Entity" in curr_pred.split('[')[-1]:
#                     retrieval_results = {}
#                     preds, scores, next_entities, contexts = run_entity_generation_batch(
#                         model, cur_prompt, topic_entity, context)
#                     for i, (pred, p_score,next_topic, context) in enumerate(zip(preds, scores, next_entities, contexts)):
#                         retrieval_results[i] = {
#                             "pred": pred, "score": p_score, "next_topic": next_topic, "context": context}

#                     for i, result in retrieval_results.items():
#                         node_id += 1
#                         node_score = result["score"] * \
#                             score if score is not None else result["score"]
#                         pred = result["pred"]
#                         next_entity = result['next_topic']
#                         if len(next_entity) == 0:
#                             next_entity = topic_entity
#                         prediction_tree[node_id] = {"prompt": cur_prompt, "pred": pred, "context": result['context'],
#                                                     "score": node_score, "parent": node,
#                                                     "topic_entity": next_entity}
#                         if "[Continue to Retrieve Evidence]" in pred:
#                             gen_result_index = pred.index("[Continue to Retrieve Evidence]")
#                             prev_generation = pred[:gen_result_index]
#                         else:
#                             prev_generation = pred
#                         prediction_tree[node_id]["processed_pred"] = prev_generation
#                         levels[curr_depth].append(node_id)
#                 #存在前后逻辑粘连   
#                 if "New Retrieval" in curr_pred.split('[')[-1] or "Continue to Retrieve Evidence" in curr_pred.split('[')[-1]:
#                     retrieval_results = {}
#                     preds, scores, contexts = run_relation_generation_batch(
#                         model, cur_prompt, new_retrieval=True if ("[New Retrieval]" in curr_pred) else False, context=context, topic_entity=topic_entity, hypo=True)
#                     for i, (pred, p_score, context) in enumerate(zip(preds, scores, contexts)):
#                         retrieval_results[i] = {
#                             "pred": pred, "score": p_score, "context": context}

#                     for i, result in retrieval_results.items():
#                         node_id += 1
#                         node_score = result["score"] * \
#                             score if score is not None else result["score"]
#                         pred = result["pred"]
#                         context = result["context"]
#                         prediction_tree[node_id] = {"prompt": cur_prompt, "pred": pred,
#                                                     "score": node_score, "parent": node,
#                                                     "topic_entity": topic_entity, "context": context}
#                         if "[Retrieve Entity]" in pred:
#                             gen_result_index = pred.index("[Retrieve Entity]")
#                             prev_generation = pred[:gen_result_index]
#                         else:
#                             prev_generation = pred
#                         prediction_tree[node_id]["processed_pred"] = prev_generation
#                         levels[curr_depth].append(node_id)
#             current_rank = levels[curr_depth]
#             node2score = {
#                 node_id: prediction_tree[node_id]["score"] for node_id in current_rank}
#             top_nodes = sorted(node2score.items(), key=lambda x: x[1], reverse=True)[
#                 :3]
#             levels[curr_depth] = [node[0] for node in top_nodes]
#             curr_depth += 1
#         else:
#             break
#     labels = [get_label(ans) if ans.startswith('m.') else ans for ans in test_data[index]['answer']]
#     # print(labels)
#     for tree_node in prediction_tree.values():
#         if 'Answer' in tree_node['processed_pred']:
#             answer = tree_node['processed_pred'].split('Answer:')[-1]
#             for label in labels:
#                 if label and label in answer:
#                     hit = 1
#     if hit == 1:
#         print('Correct')
#         count += 1
#         correct_ids.append(index)
#     else:
#         break
#         logging_res.append({"index": index, "tree": prediction_tree})
#     if len(logging_res) == 20:
#         save_to_json(logging_res, './output/inference/webqsp_test_res.json')
#         logging_res = []

#     #注意有value标签, e.g. WebQTest-31
#     # for label in labels:
#     #     if label and label in prediction_tree[len(prediction_tree)-1]['processed_pred']:
#     #         count += 1
#     #         break
#     # except:
#     #     print(f'{index} Error')
#     #     continue


Process 21


Processed prompts: 100%|██████████| 3/3 [00:00<00:00,  3.41it/s, est. speed input: 835.67 toks/s, output: 150.08 toks/s]


#### Test Part

In [47]:
import json
with open('./output/inference/cwq_test_res_1217.json', 'r', encoding='utf-8') as f:
    cwq_result = json.load(f)

In [22]:
for i, error in enumerate(result):
    if error['index'] == 14:
        print(i, error)

5 {'index': 14, 'tree': {'0': {'prompt': '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nWhen was the last time a team led by Larr Baer win the World Series?<|eot_id|><|start_header_id|>assistant<|end_header_id|>', 'pred': '[New Retrieval]', 'processed_pred': '', 'score': None, 'topic_entity': ['m.0hhv_6h'], 'parent': None, 'context': ''}, '1': {'prompt': '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nWhen was the last time a team led by Larr Baer win the World Series?<|eot_id|><|start_header_id|>assistant<|end_header_id|>', 'pred': '[New Retrieval]<paragraph>business.board_member.leader_of;organization.leadership.person;common.topic.article;people.person.gender;common.topic.description</paragraph>business.board_member.leader_of[Fully Relevant]organization.leadership.person[Partially Relevant]common.topic.article[Unrelevant][Retrieve Entity]', 'score': 1.0, 'parent': 0, 'topic_entity': ['m.0hhv_6h'], 'context': 'business.board_member.leader_of[Fully Relevan

In [72]:
cwq_result[10]

{'index': 32,
 'tree': {'0': {'prompt': '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nWhat movie with the character Teklel Hafouli did Ron Howard do?<|eot_id|><|start_header_id|>assistant<|end_header_id|>',
   'pred': '[New Retrieval]',
   'processed_pred': '',
   'score': None,
   'topic_entity': ['m.0g2lq', 'm.01064ngx'],
   'parent': None,
   'context': ''},
  '1': {'prompt': '<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nWhat movie with the character Teklel Hafouli did Ron Howard do?<|eot_id|><|start_header_id|>assistant<|end_header_id|>',
   'pred': '[New Retrieval]<paragraph>film.dubbing_performance.actor;film.film.directed_by;film.actor.film;film.performance.actor;film.film.produced_by</paragraph>film.directed_by[Fully Relevant]film.actor.film[Fully Relevant]film.film.produced_by[Partially Relevant][Retrieve Entity]',
   'score': 1.0,
   'parent': 0,
   'topic_entity': ['m.0g2lq', 'm.01064ngx'],
   'context': 'film.directed_by[Fully Relevant]film.a

In [57]:
get_another_entity('m.0dl567', 'film.actor.film')

{'m.0gwnx7m': 'm.0gwnx7m',
 'm.09kkm44': 'm.09kkm44',
 'm.0y4q1x7': 'm.0y4q1x7',
 'm.0ysr8tn': 'm.0ysr8tn',
 'm.0znq0tb': 'm.0znq0tb'}